In [1]:
import geopandas as gpd
import duckdb
from graph_tool.all import Graph, remove_parallel_edges, GraphView
from graph_tool.draw import graph_draw
from graph_tool.centrality import eigenvector
from graph_tool import load_graph, show_config
from graph_tool.topology import label_components
import numpy as np
import pandas as pd
import gzip

In [2]:
with gzip.open('../data/transmission_lines.geojson.gz', 'rt') as f:
    gdf = gpd.read_file(f)

In [3]:
exploded = gdf.explode().reset_index().drop(['level_0', 'level_1'], axis=1)

/var/folders/3n/qtttn0wx3qv7fwkq55qhm2cm0000gn/T/ipykernel_19012/3017378427.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  exploded = gdf.explode().reset_index().drop(['level_0', 'level_1'], axis=1)


In [4]:
usa = exploded.cx[-130:-40, 25:55]

In [5]:
usa['coords_one'] = usa['geometry'].apply(lambda x: ', '.join(map(str, (x.coords[0][0], -1 * x.coords[0][1])))) # We need to reverse the coords
usa['coords_two'] = usa['geometry'].apply(lambda x: ', '.join(map(str, (x.coords[-1][0], -1 * x.coords[-1][1])))) # The GeoJSON has flipped coords

/Users/cyruskirby/opt/miniconda3/envs/transmission_grid/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/cyruskirby/opt/miniconda3/envs/transmission_grid/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [6]:
usa.columns = [c.lower() for c in usa.columns]

In [7]:
sql_df = usa.drop('geometry', axis=1)

In [27]:
with duckdb.connect('transmission.db') as con:
    # con.sql('CREATE TABLE lines AS SELECT * FROM sql_df')
    pass

In [20]:
with duckdb.connect('transmission.db') as con:
    coords = con.sql("""
            WITH all_coords AS (
                SELECT 
                    objectid
                    , coords_one AS coords
                FROM 
                    usa
                UNION ALL
                SELECT 
                    objectid
                    , coords_two 
                FROM 
                    usa
                )
            
            SELECT
                objectid
                , COUNT(coords)
            FROM 
                all_coords
            GROUP BY 
                objectid
            --HAVING 
             --   COUNT(coords) = 1
            """).df()
    
    subs = con.sql("""
            WITH all_subs AS (
                SELECT 
                    objectid
                    , sub_1 AS subs
                FROM 
                   usa
                UNION ALL
                SELECT 
                    objectid
                    , sub_2 
                FROM 
                   usa
                )
            
            SELECT 
                objectid
                , COUNT(subs)
            FROM 
                all_subs
            GROUP BY 
                objectid
            --HAVING 
            --    COUNT(subs) > 4
            """).df()

In [21]:
coords

,objectid,count(coords)
0,1,2
1,12,2
2,16,2
3,17,2
4,23,2
...,...,...
93574,94198,2
93575,94200,2
93576,94201,2
93577,94205,2


In [22]:
subs

,objectid,count(subs)
0,1,2
1,12,2
2,16,2
3,17,2
4,23,2
...,...,...
93574,94174,2
93575,94175,2
93576,94192,2
93577,94211,2


In [24]:
with duckdb.connect('transmission.db') as con:
    #con.sql('CREATE TABLE usa AS SELECT * FROM sql_df')
    res = con.sql("""
        WITH lone_coords_one AS (
            SELECT
                coords_one,
                COUNT(coords_one) AS coords_one_ct
            FROM 
                usa
            GROUP BY 
                coords_one
            HAVING
                COUNT(coords_one) = 1
            ),

        lone_coords_two AS (
            SELECT
                coords_two,
                COUNT(coords_two) AS coords_two_ct
            FROM 
                usa
            GROUP BY 
                coords_two
            HAVING
                COUNT(coords_two) = 1
            ),

        coords_one_not_in_coords_two AS (
            SELECT 
                coords_one
            FROM 
                lone_coords_one c1
            LEFT JOIN
                lone_coords_two c2
            ON 
                c1.coords_one = c2.coords_two
            WHERE 
                c2.coords_two IS NULL),
        
        coords_two_not_in_coords_one AS (
            SELECT 
                coords_two
            FROM 
                lone_coords_two c2
            LEFT JOIN
                coords_one_not_in_coords_two c1
            ON 
                c2.coords_two = c1.coords_one
            WHERE 
                c1.coords_one IS NULL)

        SELECT
            *
        FROM 
            coords_two_not_in_coords_one
        """).df()

#### Goal

|objectid|coord1|coord2|

1. Find all coords that appear only once, and find the object they're attached to. (join coords to usa)
2. Find all subs that appear only once, and find the object they're attached to. (join subs to usa)
3. Find 


OR 

1. Find all coords and all subs
2. Find edges associated with those coords and subs 
3. 
4. 

### Create a vertices table

| vertex_id | coord | name | type |
|-----------|-------|------|------|
| 1 | -89.9, 30.4   | S331 | substation |

Create it from the lines_raw table by doing the following:

create coords_raw table by finding distinct coords from lines table
| coord_id | coord |
|----------|-------|
| 1  | -89.9, 30.4 |

create subs_raw table by finding distinct subs from lines table

| sub_id | sub_name |
|--------|----------|
| 1      | S331     |

Find substations associated with coords by:
1. joining coords table to lines table on coord 1, then coord 2, selecting sub 1, coord1 and sub 2, coord2
2. union the result, getting a table with distinct substation names and coords
3. ^ not quite right because some coords are phantom coords - they are ever so slightly off from their true value, and should be replaced with whatever coord existing lines have at the shared substation

Step 1: Determine substation coords:
1. join subs table to lines table twice, on sub_name = sub_1 then sub_name = sub_2, selecting coords and subs
2. exclude substations that = "NOT AVAILABLE"
3. union the result
4. select sub and the coord with most connections. If count is 1, then AVG(coords)
5. create a new subs table with this information

^ What about unknown substations ("NOT AVAILABLE") that should be repaired via coords-matching?
We want to generate new station names and coords.

Step 2: 
1. SELECT sub_1, coords_one from lines WHERE sub_1 = 'NOT AVAILABLE' UNION SELECT coords_two from lines WHERE sub_two = 'NOT AVAILABLE'

Identify and fix lines that share a substation but have different coords:





Types of vertices, in decreasing quality:
Derived from edge(s), ...
- whose substations are the same and coordinates are the same (BEST).
- some substations are nonexistent but the rest are the same, and their coordinates are the same (GOOD).
- whose substations are nonexistent but their coordinates are the same (GOOD). In this case, create a new substation with the common coords
- whose coordinates are different but share a substation (FINE). In this case, impute coords by the most common if one coord appears more than once, or the mean.
- whose coordinates are different and don't share a substation (TERRIBLE). In this case, I might have to infer that a substation exists where edges terminate within $\epsilon_1$-tolerance AND there isn't another substation within $\epsilon_2$-tolerance. This might be error-prone and unnecessary.

Create the vertices table by going down the list

In [28]:
with duckdb.connect('transmission.db') as con:
    con.sql('INSTALL spatial; LOAD spatial;')
    con.sql("""
        CREATE OR REPLACE TABLE vertices (
            id INTEGER PRIMARY KEY,
            coords POINT_2D(2),
            name VARCHAR(100),
            type VARCHAR(100), 
        );
        """)

In [127]:
with duckdb.connect('transmission.db') as con:
    con.sql("""
            CREATE TEMPORARY VIEW coords_subs_combos AS (
                SELECT
                    coords_one AS coords
                    , sub_1 AS sub
                FROM
                    usa
                UNION ALL
                SELECT
                    coords_two
                    , sub_2
                FROM
                    usa
                UNION ALL
                SELECT
                    coords_one
                    , sub_2
                FROM
                    usa
                UNION ALL
                SELECT 
                    coords_two
                    , sub_1
                FROM
                    usa
            )
                """)
    
    # This extracts 
    coords = con.sql("""
        WITH most_common_value AS (
            SELECT
                coords,
                MODE(sub) AS most_common_sub
            FROM
                coords_subs_combos
            GROUP BY
                coords
        ),
        most_common_counts AS (
            SELECT
                mcv.coords,
                mcv.most_common_sub,
                COUNT(*) FILTER (WHERE csc.sub = mcv.most_common_sub) AS most_common_count,
                COUNT(*) AS total_count
            FROM
                coords_subs_combos csc
            JOIN
                most_common_value mcv
            ON
                csc.coords = mcv.coords
            GROUP BY
                mcv.coords, mcv.most_common_sub
        )

        SELECT *
        FROM most_common_counts
        WHERE most_common_count / total_count::float > 0.5;
        """).df()
    
    # coords = con.sql('SELECT * FROM coords_subs_combos').df()
    
    # # finds all the lines that terminate in a bad coord
    # res1 = con.sql("""
    #     WITH lone_coords AS (
    #         SELECT 
    #             coords
    #             , COUNT(coords)
    #         FROM 
    #             distinct_coords_subs
    #         GROUP BY
    #             coords
    #         HAVING
    #             COUNT(coords) = 1
    #     )
            
    #         SELECT DISTINCT 
    #             lines.*
    #         FROM
    #             lone_coords lcrds
    #         LEFT JOIN 
    #             lines       
    #         ON 
    #             lcrds.coords = lines.coords_one
    #         OR 
    #             lcrds.coords = lines.coords_two
    #         """).df()
    
    # res2 = con.sql("""
    #         WITH lone_subs AS (
    #         SELECT 
    #             sub
    #             , COUNT(sub)
    #         FROM 
    #             distinct_coords_subs
    #         GROUP BY
    #             sub
    #         HAVING
    #             COUNT(sub) = 1
    #     )
            
    #         SELECT DISTINCT 
    #             lines.*
    #         FROM
    #             lone_subs lsubs
    #         LEFT JOIN 
    #             lines       
    #         ON 
            #     lsubs.sub = lines.sub_1
            # OR 
            #     lsubs.sub = lines.sub_2
            # """).df()
            

In [ ]:
with duckdb.connect('transmission.db') as con:
    con.sql("""
        


In [130]:
filter_coords(usa, '-83.7558314, -36.7951748')

,objectid,id,type,status,naics_code,naics_desc,source,sourcedate,val_method,val_date,...,voltage,volt_class,inferred,sub_1,sub_2,globalid,shape_length,geometry,coords_one,coords_two
4418,4418,146458,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, http://www.oatioasis.com/TVA/TVAdocs/...",2015-03-25 00:00:00+00:00,IMAGERY,2015-10-08 00:00:00+00:00,...,345.0,345,Y,UNKNOWN109464,PINEVILLE,{1F3DF40D-CD3E-432B-B7D7-68453BC1C0BD},0.844046,"LINESTRING (-83.75583 36.79517, -83.75622 36.7...","-83.7558314, -36.7951748","-84.537962, -37.0463413"
7149,7149,150162,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 861, EIA 860",2015-03-25 00:00:00+00:00,IMAGERY,2019-02-05 00:00:00+00:00,...,161.0,100-161,Y,TAP153142,PINEVILLE,{26FEA697-8755-453D-BFBC-970D4F48BCF9},0.246104,"LINESTRING (-83.51456 36.83570, -83.51872 36.8...","-83.5145564, -36.8357002","-83.7558314, -36.7951748"
7938,7938,155813,OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,IMAGERY,2018-08-20 00:00:00+00:00,IMAGERY,2018-08-20 00:00:00+00:00,...,-999999.0,NOT AVAILABLE,N,UNKNOWN121650,PINEVILLE,{CB7D3815-71C5-4A19-8A73-7F19C127505C},0.002733,"LINESTRING (-83.75672 36.79726, -83.75685 36.7...","-83.7567241, -36.7972649","-83.7558314, -36.7951748"
10271,10271,158987,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, http://psc.ky.gov/pscecf/2016-00370/d...",2018-08-22 00:00:00+00:00,IMAGERY,2019-02-08 00:00:00+00:00,...,69.0,UNDER 100,Y,UNKNOWN121648,PINEVILLE,{4312B366-7465-46E5-865B-40CA6664F9F4},0.076677,"LINESTRING (-83.69403 36.75800, -83.69425 36.7...","-83.694034, -36.758002","-83.7558314, -36.7951748"
16071,16070,122132,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 861, http://psc.ky.gov/pscecf/201...",2015-03-25 00:00:00+00:00,IMAGERY,2019-02-08 00:00:00+00:00,...,69.0,UNDER 100,Y,UNKNOWN121649,PINEVILLE,{0E3395A5-AD87-497B-A0A9-BF5030E49078},0.039827,"LINESTRING (-83.71715 36.79708, -83.73236 36.7...","-83.717154, -36.7970788","-83.7558314, -36.7951748"
16074,16073,134430,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 861, EIA 860",2015-03-24 00:00:00+00:00,IMAGERY,2019-02-05 00:00:00+00:00,...,161.0,100-161,Y,HARLAN Y,PINEVILLE,{D6F85880-64DE-4842-A715-EFD375EA9DB2},0.450310,"LINESTRING (-83.32644 36.84314, -83.32662 36.8...","-83.3264384, -36.8431368","-83.7558314, -36.7951748"
16081,16080,134915,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 860",2016-09-27 00:00:00+00:00,IMAGERY,2019-02-05 00:00:00+00:00,...,500.0,500,Y,POCKET NORTH,PINEVILLE,{BD94DFF2-8A39-4D88-A4A4-0756080D7D2B},0.726497,"LINESTRING (-83.75583 36.79517, -83.75516 36.7...","-83.7558314, -36.7951748","-83.0433244, -36.7839953"
16180,16179,120467,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 861, http://psc.ky.gov/pscecf/201...",2015-03-25 00:00:00+00:00,IMAGERY,2019-02-08 00:00:00+00:00,...,69.0,UNDER 100,Y,PINEVILLE,PINEVILLE,{E9739798-39DD-4E52-80AB-DDF82EF49D85},0.004491,"LINESTRING (-83.75583 36.79517, -83.75560 36.7...","-83.7558314, -36.7951748","-83.7581113, -36.7976789"
16930,16929,164390,OVERHEAD,NOT AVAILABLE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, http://psc.ky.gov/pscecf/2016-00370/d...",2016-09-27 00:00:00+00:00,UNVERIFIED,2019-02-08 00:00:00+00:00,...,69.0,UNDER 100,Y,PINEVILLE,TAP160529,{85EBE1D3-2ACD-455C-9E02-0A8F4690231C},0.178247,"LINESTRING (-83.57903 36.78630, -83.58079 36.7...","-83.579034, -36.786304","-83.7558314, -36.7951748"
22376,22375,106400,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, http://www.oatioasis.com/TVA/TVAdocs/...",2014-09-19 00:00:00+00:00,IMAGERY,2015-10-08 00:00:00+00:00,...,345.0,345,Y,UNKNOWN109464,PINEVILLE,{0B6DBE03-976A-4BAB-B6E2-A4E46DFFD7AE},0.843530,"LINESTRING (-83.75583 36.795

In [129]:
coords

,coords,most_common_sub,most_common_count,total_count
0,"-83.7558314, -36.7951748",PINEVILLE,11,20
1,"-80.1655733, -27.0474144",HOBE,6,10
2,"-76.7231473, -40.1536601",THREE MILE ISLAND,8,14
3,"-88.3471406, -41.3527523",COLLINS,4,6
4,"-72.1679348, -41.3105349",MILLSTONE,7,8
...,...,...,...,...
945,"-121.0907025, -39.0438941",NOT AVAILABLE,4,4
946,"-120.1260427, -39.7959555",NOT AVAILABLE,2,2
947,"-120.1414116, -40.0541702",NOT AVAILABLE,2,2
948,"-120.1645547, -40.0789039",NOT AVAILABLE,2,2


In [105]:
filter_coords(usa, '-86.294492, -33.157928')

,objectid,id,type,status,naics_code,naics_desc,source,sourcedate,val_method,val_date,...,voltage,volt_class,inferred,sub_1,sub_2,globalid,shape_length,geometry,coords_one,coords_two
45,46,164230,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 860",2019-03-29 00:00:00+00:00,IMAGERY,2019-03-29 00:00:00+00:00,...,115.0,100-161,Y,UNKNOWN162137,TAP162138,{94D8C5F3-F48F-44B1-BD0A-0CD70D52EFEA},0.001750,"LINESTRING (-86.29621 33.15757, -86.29449 33.1...","-86.2962056, -33.1575742","-86.294492, -33.157928"
48,49,164231,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 861",2019-03-29 00:00:00+00:00,IMAGERY,2020-04-13 00:00:00+00:00,...,115.0,100-161,Y,TAP146028,TAP162138,{85D94E74-7CB4-4BDB-A049-E8920C3C34DA},0.012133,"LINESTRING (-86.30301 33.15375, -86.29901 33.1...","-86.303006, -33.1537472","-86.294492, -33.157928"
10611,10611,165970,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 861",2014-04-07 00:00:00+00:00,IMAGERY,2019-03-29 00:00:00+00:00,...,115.0,100-161,Y,AMEA PEAKING,TAP162138,{86D95518-71AA-4F82-A10E-7EC616B2B416},0.014641,"LINESTRING (-86.29449 33.15793, -86.29473 33.1...","-86.294492, -33.157928","-86.2832612, -33.1664276"


In [115]:
coords[coords['coords'] == '-86.294492, -33.157928']

,coords,sub
10356,"-86.294492, -33.157928",AMEA PEAKING
93847,"-86.294492, -33.157928",TAP162138
93850,"-86.294492, -33.157928",TAP162138
197960,"-86.294492, -33.157928",TAP162138
281451,"-86.294492, -33.157928",UNKNOWN162137
281454,"-86.294492, -33.157928",TAP146028


In [114]:
coords[coords['sub'] == 'TAP162138']

,coords,sub
93847,"-86.294492, -33.157928",TAP162138
93850,"-86.294492, -33.157928",TAP162138
104158,"-86.2832612, -33.1664276",TAP162138
187649,"-86.2962056, -33.1575742",TAP162138
187652,"-86.303006, -33.1537472",TAP162138
197960,"-86.294492, -33.157928",TAP162138


In [107]:
coords[coords['coords'] == '-86.294492, -33.157928']

,coords,sub
10356,"-86.294492, -33.157928",AMEA PEAKING
152922,"-86.294492, -33.157928",TAP162138
173574,"-86.294492, -33.157928",TAP162138
293796,"-86.294492, -33.157928",UNKNOWN162137
314100,"-86.294492, -33.157928",TAP146028


In [108]:
coords

,coords,sub
0,"-70.9285537, -42.54359",WATERS RIVER
1,"-82.961231, -36.3769931",PHILLIPS BEND
2,"-75.3340663, -43.0894616",TAP140359
3,"-89.9372786, -30.0090511",GENTILLY ROAD
4,"-90.5660876, -41.6125604",UNKNOWN137689
...,...,...
334340,"-113.1277115, -33.5223118",UNKNOWN305697
334341,"-110.3257844, -34.4984209",UNKNOWN300496
334342,"-110.3248727, -34.4990385",UNKNOWN300496
334343,"-114.5746014, -32.7815718",NOT AVAILABLE


In [118]:
filter_coords(usa, '-99.998655, -48.165811')

,objectid,id,type,status,naics_code,naics_desc,source,sourcedate,val_method,val_date,...,voltage,volt_class,inferred,sub_1,sub_2,globalid,shape_length,geometry,coords_one,coords_two
65004,64998,167385,OVERHEAD,NOT AVAILABLE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, IMAGERY, https://www.transmission.xce...",2016-11-22 00:00:00+00:00,IMAGERY,2021-02-25 00:00:00+00:00,...,115.0,100-161,Y,TAP166963,TAP166966,{F0ED53CA-83EA-41FD-B538-D1282463D552},0.194850,"LINESTRING (-99.99865 48.16581, -99.99865 48.1...","-99.998655, -48.165811","-99.934382, -48.03576"
65037,65031,168007,OVERHEAD,NOT AVAILABLE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, IMAGERY, https://www.transmission.xce...",2016-11-22 00:00:00+00:00,IMAGERY,2021-02-25 00:00:00+00:00,...,115.0,100-161,Y,TAP166963,TAP166964,{FF03CCE0-5E0F-49C4-A0D6-5E687E6569F6},0.176513,"LINESTRING (-99.99843 48.34069, -99.99845 48.3...","-99.998428, -48.340689","-99.998655, -48.165811"
65038,65032,168160,OVERHEAD,NOT AVAILABLE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, IMAGERY, https://www.transmission.xce...",2019-08-23 00:00:00+00:00,IMAGERY,2021-02-25 00:00:00+00:00,...,115.0,100-161,Y,UNKNOWN134400,TAP166963,{ADBBAE63-B086-48BC-B896-FE574A5ED897},0.000643,"LINESTRING (-99.99865 48.16581, -99.99802 48.1...","-99.998655, -48.165811","-99.9980176, -48.1657268"


In [116]:
coords.groupby('coords').agg(pd.Series.mode)

,sub
coords,
"-100.0003858, -35.175391","[FLOYD, TAP311608]"
"-100.0012705, -48.3739498","[TAP134366, UNKNOWN134346]"
"-100.005142, -40.176327","[TAP168013, UNKNOWN157212]"
"-100.005379, -40.175994",TAP168013
"-100.0063216, -37.7232757",UNKNOWN118023
...,...
"-99.9977626, -48.3406305","[TAP166964, UNKNOWN134374]"
"-99.9979855, -48.3523122",UNKNOWN134375
"-99.9980176, -48.1657268","[TAP166963, UNKNOWN134400]"


In [94]:
filter_coords(usa, '-86.294492, -33.157928')

,objectid,id,type,status,naics_code,naics_desc,source,sourcedate,val_method,val_date,...,voltage,volt_class,inferred,sub_1,sub_2,globalid,shape_length,geometry,coords_one,coords_two
45,46,164230,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 860",2019-03-29 00:00:00+00:00,IMAGERY,2019-03-29 00:00:00+00:00,...,115.0,100-161,Y,UNKNOWN162137,TAP162138,{94D8C5F3-F48F-44B1-BD0A-0CD70D52EFEA},0.001750,"LINESTRING (-86.29621 33.15757, -86.29449 33.1...","-86.2962056, -33.1575742","-86.294492, -33.157928"
48,49,164231,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 861",2019-03-29 00:00:00+00:00,IMAGERY,2020-04-13 00:00:00+00:00,...,115.0,100-161,Y,TAP146028,TAP162138,{85D94E74-7CB4-4BDB-A049-E8920C3C34DA},0.012133,"LINESTRING (-86.30301 33.15375, -86.29901 33.1...","-86.303006, -33.1537472","-86.294492, -33.157928"
10611,10611,165970,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 861",2014-04-07 00:00:00+00:00,IMAGERY,2019-03-29 00:00:00+00:00,...,115.0,100-161,Y,AMEA PEAKING,TAP162138,{86D95518-71AA-4F82-A10E-7EC616B2B416},0.014641,"LINESTRING (-86.29449 33.15793, -86.29473 33.1...","-86.294492, -33.157928","-86.2832612, -33.1664276"


In [92]:
coord

,coords,most_common_sub
0,"-86.294492, -33.157928",AMEA PEAKING
1,"-71.4834225, -42.8305411",UNKNOWN132735
2,"-81.48307, -31.1363672",UNKNOWN111827
3,"-83.8378399, -42.0226736",PARR ROAD
4,"-83.5835958, -36.7960574",UNKNOWN109420
...,...,...
74194,"-120.0411967, -39.9462564",NOT AVAILABLE
74195,"-120.1105401, -40.0300119",NOT AVAILABLE
74196,"-116.9727194, -32.7952528",EL CAJON
74197,"-121.7898222, -38.0049219",TAP311633


,objectid,id,type,status,naics_code,naics_desc,source,sourcedate,val_method,val_date,...,voltage,volt_class,inferred,sub_1,sub_2,globalid,shape_length,geometry,coords_one,coords_two
0,1,100511,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 861, EIA 860",2016-11-08 00:00:00+00:00,IMAGERY,2017-03-27 00:00:00+00:00,...,115.0,100-161,Y,WATERS RIVER,TAP136228,{F1500A3A-804A-4DA4-B6F6-BD0FBB2CF36D},0.000535,"LINESTRING (-70.92855 42.54359, -70.92841 42.5...","-70.9285537, -42.54359","-70.9284093, -42.5441055"
1,2,140809,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 860",2014-04-16 00:00:00+00:00,IMAGERY,2017-02-15 00:00:00+00:00,...,161.0,100-161,Y,PHILLIPS BEND,JOHN SEVIER,{58C24E54-D758-4071-9EBF-D83BEB60C707},0.198755,"LINESTRING (-82.96123 36.37699, -82.96090 36.3...","-82.961231, -36.3769931","-82.8078841, -36.4607057"
2,3,140837,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, https://www9.nationalgridus.com/oasis...",2015-06-16 00:00:00+00:00,IMAGERY,2019-03-05 00:00:00+00:00,...,115.0,100-161,Y,TAP140359,TAP140373,{C40BBFF1-B1ED-4905-9FF6-1A80B7B422FC},0.042609,"LINESTRING (-75.33407 43.08946, -75.33426 43.0...","-75.3340663, -43.0894616","-75.3444945, -43.1239924"
3,4,140811,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, OpenStreetMap",2014-06-20 00:00:00+00:00,IMAGERY,2017-03-20 00:00:00+00:00,...,115.0,100-161,Y,GENTILLY ROAD,MICHOUD STATION,{F09483E3-5419-4B94-AAC5-A6450F2452C4},0.120183,"LINESTRING (-89.93728 30.00905, -89.93773 30.0...","-89.9372786, -30.0090511","-89.8974078, -30.0389191"
4,5,140813,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 860",2016-10-04 00:00:00+00:00,IMAGERY/OTHER,2018-05-09 00:00:00+00:00,...,161.0,100-161,Y,UNKNOWN137689,TAP137690,{36AC4F5E-AD18-47C2-BB7D-8BA64B7FA822},0.069155,"LINESTRING (-90.56609 41.61256, -90.56468 41.6...","-90.5660876, -41.6125604","-90.529538, -41.644519"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94434,94212,313969,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,IMAGERY,2018-08-28 00:00:00+00:00,IMAGERY/OTHER,2022-05-24 00:00:00+00:00,...,69.0,UNDER 100,N,TAP302103,STEWART,{E2D37FB1-DCA0-406F-9033-D692F5F4BC99},0.110071,"LINESTRING (-109.87545 32.23874, -109.87542 32...","-109.875452, -32.238737","-109.8926162, -32.3408113"
94435,94213,313970,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,IMAGERY,2018-10-16 00:00:00+00:00,IMAGERY/OTHER,2022-05-24 00:00:00+00:00,...,69.0,UNDER 100,N,TAP300244,HACKBERRY,{3C2F7C5C-728D-4CE3-B663-C2555474B534},0.218612,"LINESTRING (-109.60459 32.87502, -109.61015 32...","-109.6045856, -32.8750173","-109.7821446, -32.84572"
94436,94214,202495,AC; OVERHEAD,NOT AVAILABLE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,IMAGERY,2019-04-10 00:00:00+00:00,IMAGERY,2019-04-10 00:00:00+00:00,...,115.0,100-161,N,TAP204261,UNKNOWN202234,{00225744-4CF6-4F33-9D8A-6F458515CAD8},0.891829,"LINESTRING (-104.45345 42.05791, -104.45308 42...","-104.4534475, -42.0579095","-103.6625547, -41.8312053"
94437,94215,202496,AC; OVERHEAD,NOT AVAILABLE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,IMAGERY,2019-04-10 00:00:00+00:00,IMAGERY,2019-04-10 00:00:00+00:00,...,115.0,100-161,N,TAP161754,TAP204258,{17D49594-E00F-4A93-B602-FE5992196552},0.422178,"LINESTRING (-103.87027 41.81278, -103.87065 41...","-103.8702722, -41.8127768","-104.2652238, -41.8961515"


In [70]:
with duckdb.connect('transmission.db') as con:
    t = con.sql("SELECT * FROM lines WHERE coords_one = '-74.3956693, -40.4914503' OR coords_two = '-74.3956693, -40.4914503'" ).df()

In [73]:
usa[usa['objectid'].isin([139,26391])]

,objectid,id,type,status,naics_code,naics_desc,source,sourcedate,val_method,val_date,...,voltage,volt_class,inferred,sub_1,sub_2,globalid,shape_length,geometry,coords_one,coords_two
138,139,140978,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, OpenStreetMap",2015-09-10 00:00:00+00:00,IMAGERY,2017-03-27 00:00:00+00:00,...,138.0,100-161,Y,EDISON,BRUNSWICK,{F0FE24FE-2DA6-49D6-BDB6-24E35E3B3694},0.082355,"LINESTRING (-74.39567 40.49145, -74.39712 40.4...","-74.3956693, -40.4914503","-74.4614307, -40.4461503"
26393,26391,111646,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 860",2015-09-10 00:00:00+00:00,IMAGERY,2017-03-27 00:00:00+00:00,...,138.0,100-161,Y,MEADOW ROAD,EDISON,{74D474FE-6536-4F51-A38D-B00675D1F54F},0.007512,"LINESTRING (-74.39567 40.49145, -74.39579 40.4...","-74.3956693, -40.4914503","-74.3912862, -40.4969803"


In [71]:
t

,objectid,id,type,status,naics_code,naics_desc,source,sourcedate,val_method,val_date,owner,voltage,volt_class,inferred,sub_1,sub_2,globalid,shape_length,coords_one,coords_two
0,139,140978,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, OpenStreetMap",2015-09-09 20:00:00-04:00,IMAGERY,2017-03-26 20:00:00-04:00,PUBLIC SERVICE ELEC & GAS CO,138.0,100-161,Y,EDISON,BRUNSWICK,{F0FE24FE-2DA6-49D6-BDB6-24E35E3B3694},0.082355,"-74.3956693, -40.4914503","-74.4614307, -40.4461503"
1,26391,111646,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 860",2015-09-09 20:00:00-04:00,IMAGERY,2017-03-26 20:00:00-04:00,PUBLIC SERVICE ELEC & GAS CO,138.0,100-161,Y,MEADOW ROAD,EDISON,{74D474FE-6536-4F51-A38D-B00675D1F54F},0.007512,"-74.3956693, -40.4914503","-74.3912862, -40.4969803"
2,26373,111626,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, OpenStreetMap",2015-09-07 20:00:00-04:00,IMAGERY,2017-03-26 20:00:00-04:00,PUBLIC SERVICE ELEC & GAS CO,138.0,100-161,Y,MEADOW ROAD,EDISON,{61FAAC4D-496C-4776-A310-FAD7731E424D},0.007844,"-74.3912862, -40.4969803","-74.3956693, -40.4914503"
3,27394,113003,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 860",2015-09-07 20:00:00-04:00,IMAGERY,2017-03-26 20:00:00-04:00,PUBLIC SERVICE ELEC & GAS CO,138.0,100-161,Y,UNKNOWN132323,EDISON,{A7FDE603-E047-4C6B-8948-F6CADA3715F3},0.189297,"-74.5318985, -40.3679659","-74.3956693, -40.4914503"
4,31769,118785,AC; OVERHEAD,NOT AVAILABLE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 860",2017-03-26 20:00:00-04:00,IMAGERY,2017-03-26 20:00:00-04:00,PUBLIC SERVICE ELEC & GAS CO,138.0,100-161,Y,EDISON,NOT AVAILABLE,{6BC88739-28EE-4439-BE0B-00F2221038CB},0.000653,"-74.3950932, -40.4911428","-74.3956693, -40.4914503"


In [61]:
usa[(usa['sub_1'] == 'VAN BUREN') | (usa['sub_2'] == 'VAN BUREN')]

,objectid,id,type,status,naics_code,naics_desc,source,sourcedate,val_method,val_date,...,voltage,volt_class,inferred,sub_1,sub_2,globalid,shape_length,geometry,coords_one,coords_two
7453,7453,150607,OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, OpenStreetMap, EIA 861",2014-04-16 00:00:00+00:00,IMAGERY,2015-01-27 00:00:00+00:00,...,-999999.0,NOT AVAILABLE,N,METRO CENTER,VAN BUREN,{6A5D5156-ADCD-4CDD-9F12-F5314309DE24},0.029102,"LINESTRING (-86.80275 36.18742, -86.80249 36.1...","-86.8027453, -36.1874166","-86.7867679, -36.1808827"
7558,7558,150730,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, OpenStreetMap, EIA 860",2015-05-08 00:00:00+00:00,IMAGERY,2018-01-24 00:00:00+00:00,...,138.0,100-161,Y,VAN BUREN,DESOTO,{109B1AC4-A83F-444A-9D18-EDED3504C529},0.574922,"LINESTRING (-85.30567 40.23677, -85.30650 40.2...","-85.3056717, -40.2367707","-85.5285538, -40.6162"
13024,13024,118008,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, http://www.apscservices.info/RcvdDocs...",2016-08-17 00:00:00+00:00,IMAGERY,2018-11-16 00:00:00+00:00,...,161.0,100-161,Y,VAN BUREN,TAP149418,{F115BA02-EA66-4D0A-86BA-9A58A1FD94F4},0.003332,"LINESTRING (-94.35325 35.44768, -94.35357 35.4...","-94.3532515, -35.4476825","-94.3536092, -35.4443698"
20761,20760,104263,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, OpenStreetMap, EIA 860",2015-04-08 00:00:00+00:00,IMAGERY,2018-02-19 00:00:00+00:00,...,138.0,100-161,Y,HUNTINGTON JUNCTION,VAN BUREN,{C6C7EF57-5FCC-4295-A630-770011031E15},0.307848,"LINESTRING (-85.52855 40.61620, -85.52883 40.6...","-85.5285538, -40.6162","-85.3837022, -40.883323"
42374,42370,131688,OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, OpenStreetMap, EIA 861",2014-04-16 00:00:00+00:00,IMAGERY,2015-01-27 00:00:00+00:00,...,-999999.0,NOT AVAILABLE,N,VAN BUREN,CENTRAL,{28BE7BEE-6120-4E62-827E-8532319DB946},0.014696,"LINESTRING (-86.77904 36.16896, -86.77921 36.1...","-86.7790442, -36.1689611","-86.7867679, -36.1808827"
83169,83084,302808,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,https://cecgis-caenergy.opendata.arcgis.com/,2018-04-25 00:00:00+00:00,IMAGERY/OTHER,2018-04-25 00:00:00+00:00,...,92.0,UNDER 100,N,VAN BUREN,COACHELLA CITY,{31665EAD-C3D7-4736-9E29-D240BEE1C53F},0.060234,"LINESTRING (-116.19865 33.71881, -116.19891 33...","-116.1986519, -33.7188102","-116.1724212, -33.6765926"
93769,93561,313313,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, DVOF",2018-02-01 00:00:00+00:00,IMAGERY/OTHER,2020-02-05 00:00:00+00:00,...,69.0,UNDER 100,N,NOT AVAILABLE,VAN BUREN,{58672D36-EF66-42C9-B28F-F6D0093EAA67},0.051916,"LINESTRING (-101.80421 35.21448, -101.80496 35...","-101.8042059, -35.214478","-101.8386895, -35.2171117"
93771,93563,313315,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, DVOF",2019-04-01 00:00:00+00:00,IMAGERY/OTHER,2020-02-05 00:00:00+00:00,...,69.0,UNDER 100,N,VAN BUREN,NOT AVAILABLE,{7AC557EF-C64E-4003-834C-3FB0133E56A4},0.036988,"LINESTRING (-101.83869 35.21711, -101.83876 35...","-101.8386895, -35.2171117","-101.8042059, -35.214478"


In [54]:
res2

,sub,count(coords)
0,NORTHWEST JACKSON,1
1,UNKNOWN117461,1
2,UNKNOWN137979,1
3,OLD LUDLOW,1
4,UNKNOWN124594,1
...,...,...
31618,MESA HEIGHTS,1
31619,CREST,1
31620,VILLAGE,1
31621,TAP311325,1


In [55]:
res1

,objectid,id,type,status,naics_code,naics_desc,source,sourcedate,val_method,val_date,owner,voltage,volt_class,inferred,sub_1,sub_2,globalid,shape_length,coords_one,coords_two
0,1634,142845,OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,IMAGERY,2016-10-06 20:00:00-04:00,IMAGERY/OTHER,2016-10-06 20:00:00-04:00,NOT AVAILABLE,-999999.0,NOT AVAILABLE,N,MELBOURNE,APOLLO,{83291EC5-B340-4F31-AD58-59917254F628},0.056983,"-80.6090821, -28.0845805","-80.6353485, -28.1145044"
1,187,141033,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 861, EIA 860",2014-05-06 20:00:00-04:00,IMAGERY,2017-11-01 20:00:00-04:00,ALABAMA POWER CO,115.0,100-161,Y,UNKNOWN109853,TAP145922,{E3343F96-B1F1-4B0F-A6F3-5B89B38B4744},0.034231,"-85.4804705, -32.6627004","-85.5144587, -32.6630646"
2,1082,102690,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, OpenStreetMap, EIA 860",2014-06-17 20:00:00-04:00,IMAGERY,2020-04-14 20:00:00-04:00,NOT AVAILABLE,138.0,100-161,Y,UNKNOWN112461,UNKNOWN112462,{3610CD1B-CA94-4160-861C-F333ECD1E934},0.130908,"-92.2912292, -30.4440759","-92.2625002, -30.3634121"
3,1351,142505,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 860",2014-07-08 20:00:00-04:00,IMAGERY,2017-11-08 19:00:00-05:00,"DUKE ENERGY CAROLINAS, LLC",100.0,100-161,Y,UNKNOWN114266,UNKNOWN114267,{BA306F45-4489-44ED-8F16-01036E789AD8},0.037699,"-80.7418978, -35.2047552","-80.7675024, -35.1849544"
4,2786,144288,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 860",2016-11-02 20:00:00-04:00,IMAGERY/OTHER,2017-03-07 19:00:00-05:00,SOUTH CAROLINA ELECTRIC&GAS COMPANY,115.0,100-161,Y,TAP147136,UNKNOWN147137,{078198B7-F99B-4820-825C-9CA774FF110C},0.004990,"-79.942759, -32.807943","-79.9454039, -32.8054081"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46506,89304,309043,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, OpenStreetMap",2017-05-07 20:00:00-04:00,IMAGERY,2017-05-07 20:00:00-04:00,"CENTERPOINT ENERGY HOUSTON ELECTRIC, LLC",138.0,100-161,N,TAP303649,KEMAH,{E3F0D6FD-2112-4CCA-A4AE-FE136C0086FC},0.000377,"-95.026274, -29.5378983","-95.0264684, -29.538167"
46507,91696,311441,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, OpenStreetMap",2017-05-07 20:00:00-04:00,IMAGERY,2017-05-07 20:00:00-04:00,"CENTERPOINT ENERGY HOUSTON ELECTRIC, LLC",138.0,100-161,N,NORMANDY,TAP303581,{CCCB1437-FC84-4F84-B786-F14D710F9809},0.032258,"-95.2056137, -29.7694363","-95.2173317, -29.7485147"
46508,92268,312014,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,https://cecgis-caenergy.opendata.arcgis.com/,2018-04-24 20:00:00-04:00,IMAGERY/OTHER,2018-04-24 20:00:00-04:00,PACIFIC GAS & ELECTRIC COMPANY,60.0,UNDER 100,N,BALFOUR,TAP311633,{4FDA24F5-13BB-4964-8475-6B47A015F03D},0.191433,"-121.7898222, -38.0049219","-121.6783012, -37.9258516"
46509,92816,312562,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,https://cecgis-caenergy.opendata.arcgis.com/,2018-04-24 20:00:00-04:00,IMAGERY/OTHER,2018-04-24 20:00:00-04:00,MODESTO IRRIGATION DISTRICT,60.0,UNDER 100,N,POUST,ROSEMORE B,{D3A6B4D7-9326-4773-BD63-977E988A47C4},0.029621,"-121.0445726, -37.6646688","-121.0431, -37.6424424"


In [26]:
con.sql("""
        CREATE TABLE coords AS
        SELECT coords_one AS coords FROM usa
        UNION
        SELECT coords_two AS coords FROM usa
        """)


RuntimeError: Query interrupted

In [31]:
usa.head()

,objectid,id,type,status,naics_code,naics_desc,source,sourcedate,val_method,val_date,...,voltage,volt_class,inferred,sub_1,sub_2,globalid,shape_length,geometry,coords_one,coords_two
0,1,100511,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 861, EIA 860",2016-11-08 00:00:00+00:00,IMAGERY,2017-03-27 00:00:00+00:00,...,115.0,100-161,Y,WATERS RIVER,TAP136228,{F1500A3A-804A-4DA4-B6F6-BD0FBB2CF36D},0.000535,"LINESTRING (-70.92855 42.54359, -70.92841 42.5...","-70.9285537, -42.54359","-70.9284093, -42.5441055"
1,2,140809,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 860",2014-04-16 00:00:00+00:00,IMAGERY,2017-02-15 00:00:00+00:00,...,161.0,100-161,Y,PHILLIPS BEND,JOHN SEVIER,{58C24E54-D758-4071-9EBF-D83BEB60C707},0.198755,"LINESTRING (-82.96123 36.37699, -82.96090 36.3...","-82.961231, -36.3769931","-82.8078841, -36.4607057"
2,3,140837,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, https://www9.nationalgridus.com/oasis...",2015-06-16 00:00:00+00:00,IMAGERY,2019-03-05 00:00:00+00:00,...,115.0,100-161,Y,TAP140359,TAP140373,{C40BBFF1-B1ED-4905-9FF6-1A80B7B422FC},0.042609,"LINESTRING (-75.33407 43.08946, -75.33426 43.0...","-75.3340663, -43.0894616","-75.3444945, -43.1239924"
3,4,140811,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, OpenStreetMap",2014-06-20 00:00:00+00:00,IMAGERY,2017-03-20 00:00:00+00:00,...,115.0,100-161,Y,GENTILLY ROAD,MICHOUD STATION,{F09483E3-5419-4B94-AAC5-A6450F2452C4},0.120183,"LINESTRING (-89.93728 30.00905, -89.93773 30.0...","-89.9372786, -30.0090511","-89.8974078, -30.0389191"
4,5,140813,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 860",2016-10-04 00:00:00+00:00,IMAGERY/OTHER,2018-05-09 00:00:00+00:00,...,161.0,100-161,Y,UNKNOWN137689,TAP137690,{36AC4F5E-AD18-47C2-BB7D-8BA64B7FA822},0.069155,"LINESTRING (-90.56609 41.61256, -90.56468 41.6...","-90.5660876, -41.6125604","-90.529538, -41.644519"


In [ ]:
g = Graph(directed=False)

# Add vertices at where transmission lines' coords meet
vertex_pos = g.new_vertex_property("vector<double>")
vertices = {}
for v in pd.concat([usa['coords_one'], usa['coords_two']]).unique():
    v_id = g.add_vertex()
    vertices[v] = v_id
    vertex_pos[v_id] = [float(coord) for coord in v.split(",")]

g.vertex_properties['position'] = vertex_pos

# Add edges between vertices
edge_weights = g.new_edge_property("double")
edge_voltages = g.new_edge_property("double")
edge_object_id = g.new_edge_property("int")
for _, row in usa.iterrows():
    v1_id = vertices[row['coords_one']]
    v2_id = vertices[row['coords_two']]
    e = g.add_edge(v1_id, v2_id)
    edge_weights[e] = row['SHAPE_Length']
    edge_voltages[e] = row['VOLTAGE']
    edge_object_id[e] = row['OBJECTID']

# for v in pd.concat([usa['SUB_1'], usa['SUB_2']]).unique():
#     if v != 'NOT AVAILABLE':
#         v_id = g.add_vertex()
#         vertices[v] = v_id

# for _, row in usa.iterrows():    

#     v1_id = vertices[row['SUB_1']]
#     v2_id = vertices[row['SUB_2']]
#     e = g.add_edge(v1_id, v2_id)
#     edge_weights[e] = row['SHAPE_Length']
#     edge_voltages[e] = row['VOLTAGE']
#     edge_object_id[e] = row['OBJECTID']

g.edge_properties['weight'] = edge_weights
g.edge_properties['voltage'] = edge_voltages
g.edge_properties['object_id'] = edge_object_id

count appearances of coords and substations. If a coord or substation appears more than once, that means an edge can be defined between them.
If a coord appears only once, then check substations associated with the edge. Lookup all edges that have that substation in common, and retrieve the vertex that corresponds with that substation (coords_one for SUB_1 and coords_two for SUB_2)
If a substation appears only once AND the coord appears only once AND the substation is "NOT AVAILABLE", add it as a new vertex. It will probably be trimeed (by SHAPELength ?) later on anyways.

In [ ]:
counts_coords = pd.concat([usa['coords_one'], usa['coords_two']]).value_counts()
counts_subs = pd.concat([usa['SUB_1'], usa['SUB_2']]).value_counts()
coords_uniq_values = counts_coords[counts_coords == 1].index
subs_uniq_values = counts_subs[counts_subs == 1].index

In [13]:
# Create a dictionary with OBJECTID as keys and unique vertex values as values
dict_vertex = usa.groupby('OBJECTID').apply(lambda x: pd.concat([x['coords_one'], x['coords_two']]).value_counts())
dict_vertex = {k: v[v == 1].index.tolist() for k, v in dict_vertex.items()}

# Create a dictionary with OBJECTID as keys and unique sub values as values
dict_subs = usa.groupby('OBJECTID').apply(lambda x: pd.concat([x['SUB_1'], x['SUB_2']]).value_counts())
dict_subs = {k: v[v == 1].index.tolist() for k, v in dict_subs.items()}

TypeError: 'int' object is not subscriptable

In [18]:
dict_vertex.head().to_dict()

{(1, '-70.9285537, -42.54359'): 1,
 (1, '-70.9284093, -42.5441055'): 1,
 (2, '-82.961231, -36.3769931'): 1,
 (2, '-82.8078841, -36.4607057'): 1,
 (3, '-75.3340663, -43.0894616'): 1}

In [12]:
usa[['OBJECTID', 'coords_one', 'coords_two']]

,OBJECTID,vertex_one,vertex_two
0,1,"-70.9285537, -42.54359","-70.9284093, -42.5441055"
1,2,"-82.961231, -36.3769931","-82.8078841, -36.4607057"
2,3,"-75.3340663, -43.0894616","-75.3444945, -43.1239924"
3,4,"-89.9372786, -30.0090511","-89.8974078, -30.0389191"
4,5,"-90.5660876, -41.6125604","-90.529538, -41.644519"
...,...,...,...
94434,94212,"-109.875452, -32.238737","-109.8926162, -32.3408113"
94435,94213,"-109.6045856, -32.8750173","-109.7821446, -32.84572"
94436,94214,"-104.4534475, -42.0579095","-103.6625547, -41.8312053"
94437,94215,"-103.8702722, -41.8127768","-104.2652238, -41.8961515"


In [ ]:
usa[]

In [ ]:
v = usa.Parameter.value_counts()
usa[usa.Parameter.isin(v.index[v.gt(1)])]

In [ ]:
# Create a dictionary with OBJECTID as keys and unique counts_coords as values
dict_counts_coords = usa.groupby('OBJECTID').apply(lambda x: pd.concat([x['coords_one'], x['coords_two']]).value_counts())
dict_counts_coords = {k: v[v == 1].index.tolist() for k, v in dict_counts_coords.items()}

# Create a dictionary with OBJECTID as keys and unique counts_subs as values
dict_counts_subs = usa.groupby('OBJECTID').apply(lambda x: pd.concat([x['SUB_1'], x['SUB_2']]).value_counts())
dict_counts_subs = {k: v[v == 1].index.tolist() for k, v in dict_counts_subs.items()}

In [ ]:
len(coords_uniq_values), len(subs_uniq_values)

In [ ]:
coords_uniq_values

In [ ]:
g

In [ ]:
edge_object_id.a[-1]

In [ ]:
g

1. iterate over coords, adding vertices at start and end coordinates
2. iterate over coords again, connecting edges from start coord to end coord
3. iterate over substations, adding vertices at both substations
4. iterate over substations again, connecting edges from start to end substation UNLESS there is already an edge via coords
5. trim all degree one/ nodes?

In [ ]:
# Assume 'usa' is your GeoPandas DataFrame with 'SUB_1' and 'SUB_2' columns

# Create a graph
g = Graph(directed=False)

# Add vertices and vertex properties
vertex_props = g.new_vertex_property("string")
vertices = {}
for v in pd.concat([usa['coords_one'], usa['coords_two'], usa['SUB_1'], usa['SUB_2']]).unique():
    v_id = g.add_vertex()
    vertices[v] = v_id
    vertex_props[v_id] = v

g.vertex_properties['name'] = vertex_props

# Add edges
edge_weights = g.new_edge_property("double")
edge_voltages = g.new_edge_property("double")
for _, row in usa.iterrows():
    if (row['SUB_1'] != "NOT AVAILABLE") and (row['SUB_2'] != "NOT AVAILABLE"):
        v1_id = vertices.get(row['coords_one'], vertices.get(row['SUB_1']))
        v2_id = vertices.get(row['coords_two'], vertices.get(row['SUB_2']))
        
        if v1_id is not None and v2_id is not None and v1_id != v2_id:
            # Add the edge only if both vertices exist and are distinct
            e = g.add_edge(v1_id, v2_id)
            edge_weights[e] = row['SHAPE_Length']
            edge_voltages[e] = row['VOLTAGE']

g.edge_properties['weight'] = edge_weights
g.edge_properties['voltage'] = edge_voltages

In [ ]:
g # actually fewer edges than before, which seems impossible if i've done this correctly

In [ ]:
components, hist = label_components(g)

In [ ]:
unique, counts = np.unique(np.array(components.a), return_counts=True)

In [ ]:
sum(counts)

In [ ]:
counts[0]

In [ ]:
inds = counts.argsort()

In [ ]:
sorted_counts = unique[inds[::-1]]

In [ ]:
sorted_counts[100]

In [ ]:
min_component_subgraph = GraphView(g, vfilt=components.a == 1007)
min_component_subgraph

In [ ]:
min_component_subgraph.get_vertices()

In [ ]:
# ', '.join(map(str,min_component_subgraph.vp.position[47040]))
min_component_subgraph.vp.name[33297]

In [ ]:
dfs = list()
for vertex in min_component_subgraph.get_vertices():
    dfs.append(filter_coords(usa, ', '.join(map(str,min_component_subgraph.vp.name[vertex]))))

In [ ]:
dfs = list()
for vertex in min_component_subgraph.get_vertices():
    dfs.append(filter_coords(usa, min_component_subgraph.vp.name[vertex]))

In [ ]:
pd.concat(dfs, axis=0)

In [ ]:
min_component_subgraph.vp.name[16845]

In [ ]:
temp = pd.concat(dfs, axis=0).drop_duplicates()
# temp.sort_values('OBJECTID')
temp[temp['OBJECTID'] == 16845][['geometry', 'coords_one', 'coords_two']].iloc[0,0].coords[0]
temp[temp['OBJECTID'] == 16845][['geometry', 'coords_one', 'coords_two']]['geometry'].iloc[0].coords[-1]

In [ ]:
temp[temp['OBJECTID'] == 86576][['geometry', 'coords_one', 'coords_two']]

In [ ]:
usa[usa['OBJECTID'] == 90160]

In [ ]:
usa['SUB_1'].value_counts()

In [ ]:
pd.concat([usa['SUB_1'], usa['SUB_2']], axis=0).value_counts(ascending=False)

In [93]:
def filter_coords(df, coords):
    assert isinstance(coords, str)
    return df[(df['coords_one'] == coords) | (df['coords_two'] == coords)]

In [ ]:
filter_coords(usa, "-95.564805, -39.0534715")

In [ ]:
graph_draw(min_component_subgraph, pos=min_component_subgraph.vertex_properties['position'])

In [ ]:
component_views = {}
for label in range(hist.shape[0]):
    # Extract the subgraph for the current component
    subgraph = GraphView(g, vfilt=components.a == label)
    
    # Add the subgraph to the dictionary
    component_views[label] = subgraph

In [ ]:
g.vertex_properties['position'][74198]

In [ ]:
usa[(usa['coords_one'] == "-111.1079245, 34.6581138") | (usa['coords_two'] == "-111.1079245, 34.6581138")]

In [ ]:
usa[(usa['coords_one'] == "-110.6899811, 35.0229279") | (usa['coords_two'] == "-110.6899811, 35.0229279")]

In [ ]:
usa.tail()

In [ ]:
np.unique(label_components(g)[1])

In [ ]:
g_sample = sample_graph(g, proportion=.005)

In [ ]:
g = Graph()

vprop_pos = g.new_vertex_property("vector<double>")
eprop_voltage = g.new_edge_property("double")
eprop_length = g.new_edge_property("double")

# Add vertices and properties
vertices = {}
for i, row in usa.iterrows():
    v1 = row['coords_one']
    v2 = row['coords_two']
    edge = g.add_edge(v1, v2)
    vprop_pos[v1] = [float(v1[0]), float(v1[1])]
    vprop_coords_two[coords_one] = [float(v2[0]), float(v2[1])]
    vertices[i] = coords_one

# Add edges and properties
for i, row in usa.iterrows():
    coords_one = vertices[i]
    coords_two = vertices[i]
    edge = g.edge(coords_one.source(), coords_two.target())
    eprop_voltage[edge] = row['VOLTAGE']
    eprop_length[edge] = row['SHAPE_Length']

# Set properties to the graph
g.vertex_properties['coords_one'] = vprop_coords_one
g.vertex_properties['coords_two'] = vprop_coords_two
g.edge_properties['voltage'] = eprop_voltage
g.edge_properties['length'] = eprop_length

In [ ]:
data = usa[['ID', 'TYPE', 'VOLTAGE', 'VOLT_CLASS', 'SHAPE_Length', 'coords_one', 'coords_two']].to_dict()

In [ ]:
edge_list = zip(data['coords_one'].values(), data['coords_two'].values())

In [ ]:
g = Graph(edge_list, hashed=True)

In [ ]:
np.unique(label_components(g)[1])

In [ ]:
list(g.vp.ids)

In [ ]:
pos = g.new_vertex_property('vector<double>')

In [ ]:
g.get_edges()

In [ ]:
range(1, 1+len(data['coords_one']))

In [ ]:
g = Graph(directed=False)

# Define properties
vprop_coords_one = g.new_vertex_property("vector<double>")
vprop_coords_two = g.new_vertex_property("vector<double>")
eprop_voltage = g.new_edge_property("double")
eprop_length = g.new_edge_property("double")

# Add vertices and properties
vertices = {}
for i, row in df.iterrows():
    v1 = row['coords_one'].split(', ')
    v2 = row['coords_two'].split(', ')
    coords_one = g.add_edge(g.vertex(i), g.vertex(i))
    vprop_coords_one[coords_one] = [float(v1[0]), float(v1[1])]
    vprop_coords_two[coords_one] = [float(v2[0]), float(v2[1])]
    vertices[i] = coords_one

# Add edges and properties
for i, row in df.iterrows():
    coords_one = vertices[i]
    coords_two = vertices[i]
    edge = g.edge(coords_one.source(), coords_two.target())
    eprop_voltage[edge] = row['VOLTAGE']
    eprop_length[edge] = row['SHAPE_Length']

# Set properties to the graph
g.vertex_properties['coords_one'] = vprop_coords_one
g.vertex_properties['coords_two'] = vprop_coords_two
g.edge_properties['voltage'] = eprop_voltage
g.edge_properties['length'] = eprop_length

# Print the graph
print(g)

In [ ]:
list(g.vertices())

In [ ]:
for idx, row in usa.iterrows():
    geometry = row['geometry']
    if geometry.geom_type == 'MultiLineString':
        for line_string in geometry.geoms:
            # Use the first and last points as nodes
            start_node = graph.add_vertex()
            end_node = graph.add_vertex()

            # Calculate the length of the LineString and use it as the weight
            length = line_string.length

            # Add nodes and edges to the graph with weight
            pos[start_node] = line_string.coords[0]
            pos[end_node] = line_string.coords[-1]

            edge = graph.add_edge(start_node, end_node)
            weight[edge] = length
            voltage[edge] = row['VOLTAGE']
    elif geometry.geom_type == 'LineString':
        # Use the first and last points as nodes
        start_node = graph.add_vertex()
        end_node = graph.add_vertex()

        # Calculate the length of the LineString and use it as the weight
        length = geometry.length

        # Add nodes and edges to the graph with weight
        pos[start_node] = geometry.coords[0]
        pos[end_node] = geometry.coords[-1]

        edge = graph.add_edge(start_node, end_node)
        weight[edge] = length
        voltage[edge] = row['VOLTAGE']

graph.save('new_transmission_lines.gt.gz')

In [ ]:
graph.edge_properties, graph.vertex_properties

In [ ]:
graph = load_graph('new_transmission_lines.gt.gz')

In [ ]:
graph.edge_properties, graph.vertex_properties

In [ ]:
import geopandas as gpd
from shapely.geometry import MultiLineString, LineString
from graph_tool.all import Graph, graph_draw
import random


# Create an undirected graph
graph = Graph(directed=False)

# Add property for edge voltages
voltages = graph.new_edge_property("int")

# Add nodes to the graph and set initial layout positions based on GeoJSON coordinates
node_dict = {}
pos = graph.new_vertex_property("vector<double>")
graph.vp.pos = pos  # Set pos as a vertex property of the graph
graph.ep.voltage = voltages

for i, row in usa.iterrows():
    v = graph.add_vertex()
    node_dict[row['ID']] = v

    # Set initial layout position based on GeoJSON coordinates
    if isinstance(row['geometry'], MultiLineString):
        for line in list(row['geometry'].geoms):
            coords = list(line.coords)
            pos[v] = coords[0][0], -1 * coords[0][1]  # Set initial position to the first coordinate in the line
            for coord in coords[1:]:
                v = graph.vertex(node_dict[row['ID']])
                e = graph.add_edge(v, graph.vertex(node_dict[row['ID']]))
                voltages[e] = row['VOLTAGE']
    elif isinstance(row['geometry'], LineString):
        coords = list(row['geometry'].coords)
        pos[v] = coords[0][0], -1 * coords[0][1]  # Set initial position to the first coordinate in the LineString
        for coord in coords[1:]:
            v = graph.vertex(node_dict[row['ID']])
            e = graph.add_edge(v, graph.vertex(node_dict[row['ID']]))
            voltages[e] = row['VOLTAGE']

graph.save('transmission_lines.gt.gz')

In [ ]:
remove_parallel_edges(graph)

In [ ]:
graph = load_graph('new_transmission_lines.gt.gz')

In [ ]:
graph

In [ ]:
np.random.choice(graph.get_vertices())

In [ ]:
usa['geometry'].apply(lambda x: x.geom_type).value_counts()

In [ ]:
usa['geometry'].iloc[1].geoms[0].coords[0], usa['geometry'].iloc[1].geoms[0].coords[-1]

In [ ]:
usa['coords_one'], usa['coords_two'] = usa['geometry'].apply(lambda x: x.geoms[0].coords[0]), usa['geometry'].apply(lambda x: x.geoms[0].coords[-1])

In [ ]:
usa.head()

In [ ]:
import pandas as pd
subset = usa[['OBJECTID', 'VOLTAGE', 'coords_one', 'coords_two', 'geometry']]
subset.merge(right=subset, how='inner', left_on=['coords_one'], right_on=['coords_one'])

In [ ]:
merged_df = pd.merge(subset, subset, left_on=['coords_one', 'coords_two'], right_on=['coords_two', 'coords_one'], how='outer')

# Filter out rows where the vertices are the same (excluding self-connections)
result_df = merged_df[merged_df['coords_one_x'] != merged_df['coords_two_y']]

# Drop duplicate columns and reset index
result_df = result_df[['coords_one_x', 'coords_two_x']].drop_duplicates().reset_index(drop=True)

# Rename columns for clarity
result_df.columns = ['coords_one', 'coords_two']

In [ ]:
def sample_graph(graph, num_steps = None, proportion = None):
    assert num_steps or proportion
    assert not (num_steps and proportion)

    if proportion:
        num_steps = len(graph.get_vertices() * proportion)

    print(num_steps)

    edges = set()
    vertices = set()

    edge_filter = graph.new_edge_property("bool")
    edge_filter.a = False

    vertex_filter = graph.new_vertex_property("bool")
    vertex_filter.a = False

    vertex = np.random.choice(list(graph.vertices()))
    vertex_filter[vertex] = True
    vertices.add(vertex)

    for _ in range(num_steps):
        edge = random.choice(list(vertex.out_edges()))
        print(edge)
        edge_filter[edge] = True
        vertex = edge.target()
        print(vertex)
        vertex_filter[vertex] = True

        vertices.add(vertex)
        edges.add(edge)

    print(list(vertex_filter))
    print(list(edge_filter))
    
    return GraphView(graph, vfilt=vertex_filter, efilt=edge_filter)

In [ ]:
subgraph = sample_graph(graph, proportion=.05)

In [ ]:
list(subgraph.vertices())

In [ ]:
random.choice(list(vertex.out_edges())).target()

In [ ]:
vertex = random.choice(list(graph.vertices()))

In [ ]:
vertex

In [ ]:
list(vertex.out_edges())

In [ ]:
num_vertices_to_sample = int(0.05 * graph.num_vertices())

# Randomly sample vertices
sampled_vertices = random.sample(list(graph.vertices()), num_vertices_to_sample)

vertex_filter = graph.new_vertex_property("bool")
vertex_filter.a = False
for vertex in sampled_vertices:
    vertex_filter[vertex] = True

# Collect edges connected to sampled vertices
sampled_edges = set()
for vertex in sampled_vertices:
    for edge in vertex.out_edges():
        sampled_edges.add(edge)

edge_filter = graph.new_edge_property("bool")
edge_filter.a = False
for edge in sampled_edges:
    edge_filter[edge] = True

# Create a graph view with the sampled vertices and connected edges
subgraph = GraphView(graph, vfilt=vertex_filter, efilt=edge_filter)

subgraph

In [ ]:
len(sampled_edges)

In [ ]:
subgraph.save('new_sample_transmission_lines.gt.gz')

In [ ]:
subgraph = load_graph('new_sample_transmission_lines.gt.gz')

In [ ]:
graph_draw(subgraph, pos=subgraph.vp.pos, bg_color=[0,0,0,0])

In [ ]:
subgraph

In [ ]:
list(subgraph.vp.pos)

In [ ]:
eigval, eigvec = eigenvector(subgraph, subgraph.ep.voltage)

In [ ]:
show_config()